In [ ]:
!pip install -q langchain sentence-transformers cohere
!pip install langchain-community
!pip install faiss-cpu rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import FAISS
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [ ]:

import os
from getpass import getpass

HF_token = getpass()

··········


In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HF_token

In [ ]:
path='/content/drive/MyDrive/datasets/BBCNewsDataset'

#!wget -q http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip -O /content/new_papers.zip
#!unzip -q /content/new_papers.zip -d "{path}"

In [ ]:
dataset_folder_path='/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech'


In [ ]:
saved_files_count = 0
max_files_to_save = 100

for file_name in os.listdir(dataset_folder_path):
  file_path = os.path.join(dataset_folder_path, file_name)
  try:
        # Try to open the file
        with open(file_path, 'r') as file:
            file.read()  # Reading to check if it's accessible

        if saved_files_count < max_files_to_save:
          saved_files_count += 1
        else:
            # Delete the file
            os.remove(file_path)

  except Exception as e:
        # If the file can't be opened, delete it
        try:
          os.remove(file_path)
        except:
          pass

In [ ]:
documents=[]
for file_name in os.listdir(dataset_folder_path):
  if ".txt" in file_name:
    file_path = os.path.join(dataset_folder_path, file_name)
    loader=TextLoader(file_path)
    documents.extend(loader.load())

In [ ]:
documents[1]

Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/074.txt'}, page_content='China \'to overtake US net use\'\n\nThe Chinese net-using population looks set to exceed that of the US in less than three years, says a report.\n\nChina\'s net users number 100m but this represents less than 8% of the country\'s 1.3 billion people. Market analysts Panlogic predicts that net users in China will exceed the 137 million US users of the net by 2008. The report says that the country\'s culture will mean that Chinese people will use the net for very different ends than in many other nations.\n\nAlready net use in China has a very different character than in many Western nations, said William Makower, chief executive of Panlogic. In many Western nations desktop computers that can access the net are hard to escape at work. By contrast in China workplace machines are relatively rare. This, combined with the relatively high cost of PCs in China and the time it takes to 

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=512,chunk_overlap=50)
text_splits=text_splitter.split_documents(documents)
print(len(text_splits))

841


In [ ]:
embeddings=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_token,
    model_name='BAAI/bge-base-en-v1.5'
)

In [ ]:
vectorstore = FAISS.from_documents(text_splits, embeddings)

## Implementing Hybrid Search with ensemble Retrieval

In [ ]:
retriever_vectordb = vectorstore.as_retriever(search_kwargs={"k": 5})

In [ ]:
keyword_retriever = BM25Retriever.from_documents(text_splits)
keyword_retriever.k =  5

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_vectordb,keyword_retriever],
                                       weights=[0.5, 0.5])

In [ ]:
query="How many mobiles go out of useage in UK?"
docs_rel=ensemble_retriever.get_relevant_documents(query)
docs_rel

[Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/265.txt'}, page_content="Half of UK's mobiles 'go online'\n\nMultimedia mobile phones are finally showing signs of taking off, with more Britons using them to go online."),
 Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/017.txt'}, page_content='The report reveals that approximately 15 million mobile phones go out of use every year in the UK. Of the 15 million that are swapped for newer models each year, only 25% get returned to mobile phone firms for recycling or re-use. The slowly growing mass of unrecycled, discarded phones has now reached 90 million handsets, the equivalent of 9,000 tonnes of waste, estimates James Goodman, report author and a senior adviser at the Forum for the Future. "It\'s quite common for people to have two or three phones'),
 Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/265.txt'}, page_content='

## Implementing Re-ranking with Cohere-Rerank

In [ ]:
Cohere_API_token = getpass()

··········


In [ ]:
os.environ["COHERE_API_KEY"] =Cohere_API_token

In [ ]:
from langchain.llms import HuggingFaceHub
model=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-alpha',
                     model_kwargs={"temperature":0.5,"max_new_tokens":512,"max_length":64}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


In [ ]:
compressor = CohereRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
compression_retriever

ContextualCompressionRetriever(base_compressor=CohereRerank(client=<cohere.client.Client object at 0x7dd37b30ee60>, top_n=3, model='rerank-english-v2.0', cohere_api_key=None, user_agent='langchain'), base_retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7dd37bb57970>, search_kwargs={'k': 5}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7dd37bbd5d50>, k=5)], weights=[0.5, 0.5]))

In [ ]:
compressed_docs

[Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/017.txt', 'relevance_score': 0.99896777}, page_content='The report reveals that approximately 15 million mobile phones go out of use every year in the UK. Of the 15 million that are swapped for newer models each year, only 25% get returned to mobile phone firms for recycling or re-use. The slowly growing mass of unrecycled, discarded phones has now reached 90 million handsets, the equivalent of 9,000 tonnes of waste, estimates James Goodman, report author and a senior adviser at the Forum for the Future. "It\'s quite common for people to have two or three phones'),
 Document(metadata={'source': '/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/375.txt', 'relevance_score': 0.9738444}, page_content='In London alone over the past six months more than 63,000 mobile phones have been left in the back of black cabs, according to a survey. That works out at about three phones per cab. Over the same pe

## Build Chatbot Now

In [ ]:
template = """
<|system|>>
You are an AI Assistant that follows instructions extremely well.
Please be truthful and give direct answers. Please tell 'I don't know' if user query is not in CONTEXT

CONTEXT: {context}

<|user|>
{query}

<|assistant|>
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
output_parser = StrOutputParser()

In [ ]:
chain = (
    {"context": compression_retriever, "query": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
query="What is the most popular way to pay in China?"
response = chain.invoke(query)
response

'Human: \n<|system|>>\nYou are an AI Assistant that follows instructions extremely well.\nPlease be truthful and give direct answers. Please tell \'I don\'t know\' if user query is not in CONTEXT\n\nCONTEXT: [Document(metadata={\'source\': \'/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/074.txt\', \'relevance_score\': 0.99253935}, page_content=\'China partly because of fears people have about getting in to debt. "The most popular way to pay is Cash-On-Delivery," he said, "and that\\\'s quite a brake to the development of e-commerce." The arrival of foreign banks in China, due in 2006, could mean greater use of credit cards but for the moment they are rare, said Mr Makower. But if Chinese people are not spending cash online they are interested in the news they can get via the net and the view it gives them on Western ways of living. "A large part of\'), Document(metadata={\'source\': \'/content/drive/MyDrive/datasets/BBCNewsDataset/bbc/tech/063.txt\', \'relevance_score\': 0.39

In [ ]:
response.page_content